# Heating Load Analysis

KATE expects your code to define variables with specific names that correspond to certain things we are interested in.

KATE will run your notebook from top to bottom and check the latest value of those variables, so make sure you don't overwrite them.

* Remember to uncomment the line assigning the variable to your answer and don't change the variable or function names.
* Use copies of the original or previous DataFrames to make sure you do not overwrite them by mistake.

You will find instructions below about how to define each variable.

Once you're happy with your code, upload your notebook to KATE to check your feedback.

## About the data

The following dataset comes from a study about the heating load required to maintain comfortable indoor air conditions at buildings. That study investigated the effect of eight input variables (relative compactness, surface area, wall area, roof area, overall height, orientation, glazing area, glazing area distribution) on the required heating load.

This study simulated a total of 768 buildings. Those buildings had different surface areas and dimensions, but the same volume and construction materials.

The goal of this assignment is to use machine learning techniques for the prediction of the required heating load. Since the output variable of this dataset (required heating load) is a continuous variable, we will treat this problem as 

## About the assignment

A typical supervised learning task involves creating training and testing sets, performing hyperparameter tuning on the selected algorithm and validating its performance. 

A preliminary study was applied to handle this problem. The selected algorithm was *K-Nearest Neighbours* (KNN). The performance of the method was validated on the testing data by capturing two different metrics: *Mean Absolute Error* (MAE) and *Mean Square Error* (MSE). The findings of this study were:

**MAE = 1.875**\
**MSE = 8.495**

Let’s see if we can do better than that! For this assignment, we will go through the data first and check if we can apply feature engineering. Feature engineering is an important step as it can be used to create or remove features, reduce data complexity and generally understand the data better in order to improve performance. 

We will go through the steps of applying feature engineering, splitting the data into training/testing sets and using *k-fold* cross-validation to tune the parameters of KNN. The performance of the model will be validated using the **MAE** and **MSE** metrics.

## Setup

First, let's load all the necessary libraries needed for this assignment.

We will import `numpy` and `pandas` for all the data manipulation tasks. `matplotlib` and `seaborn` will be used to generate plots and graphs that will assist us during feature engineering. `scipy stats` is useful since it contains a large number of statistical functions.

Finally, `sklearn` will be used for training the regression model. For this assignment we will need modules about data preprocessing (`MinMaxScaler, OneHotEncoder`), cross-validation (`train_test_split`,`GridSearchCV`,`KFold`), metrics (`mean_absolute_error`,`mean_squared_error`) and the *KNN* regression (`KNeighborsRegressor`)

# Imports

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as stats
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import (
    MinMaxScaler,
    OneHotEncoder,
    StandardScaler,
    RobustScaler,
)
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

from typing import List, Optional

## Data

In this assignment we will load two datasets. The first one contains the data needed for training our model (let's save it to a variable called `df`). The second one contains the data needed for evaluating our model with **KATE**. This will be our held-out test data (let's save it to a variable called `df_eval`).

We will need to process `df_eval` in exactly the same way as `df`, train our model on `df` and make predictions using `df_eval`.

Run the following cell to load the data.

In [2]:
df = pd.read_csv("data/heating_load.csv")
df_eval = pd.read_csv("data/heating_load_eval.csv")

print(df.shape)  # (614, 9)
print(df_eval.shape)  # (154, 9)

(614, 9)
(154, 9)


In [3]:
df.head()

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load
0,0.66,759.5,318.5,220.5,3.5,2,0.4,3,15.16
1,0.76,661.5,416.5,122.5,7.0,3,0.1,1,32.12
2,0.66,759.5,318.5,220.5,3.5,3,0.1,1,11.69
3,0.74,686.0,245.0,220.5,3.5,5,0.1,4,10.14
4,0.64,784.0,343.0,220.5,3.5,2,0.4,4,19.06


In [4]:
df_eval.head()

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load
0,0.71,710.5,269.5,220.5,3.5,3,0.1,3,NaN
1,0.82,612.5,318.5,147.0,7.0,3,0.1,5,NaN
2,0.82,612.5,318.5,147.0,7.0,5,0.1,4,NaN
3,0.79,637.0,343.0,147.0,7.0,3,0.4,5,NaN
4,0.62,808.5,367.5,220.5,3.5,5,0.1,3,NaN


In [5]:
df_eval.describe()

,relative_compactness,surface_area,wall_area,roof_area,overall_height,orientation,glazing_area,glazing_area_distribution,heating_load
count,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,154.000000,0.0
mean,0.770390,665.477273,314.045455,175.715909,5.340909,3.519481,0.225325,2.850649,NaN
std,0.102012,85.353137,43.164390,43.942700,1.753339,1.097953,0.133975,1.472104,NaN
min,0.620000,514.500000,245.000000,110.250000,3.500000,2.000000,0.000000,0.000000,NaN
25%,0.690000,588.000000,294.000000,147.000000,3.500000,3.000000,0.100000,2.000000,NaN
50%,0.760000,661.500000,318.500000,147.000000,7.000000,3.000000,0.250000,3.000000,NaN
75%,0.860000,735.000000,343.000000,220.500000,7.000000,5.000000,0.400000,4.000000,NaN
max,0.980000,808.500000,416.500000,220.500000,7.000000,5.000000,0.400000,5.000000,NaN


Notice that while both `df` and `df_eval` have $9$ columns, the `heating_load` column in `df_eval` is full on null values.

This is because `heating_load` is our target variable and `df_eval` is our test set. In this assignment, we will use the input features from `df_eval` to predict the `heating_load`, and **KATE** will evaluate these predictions against the actual values. This is what's known as a held-out test set.

**Prepare input and output**

Prepare the input and output variables for this assignment. Store the resulting DataFrames to the following variables:

* `X_train`: the input features of `df`
* `y_train`: the output variable of `df`
* `X_eval` : the input features of `df_eval`

*Hint: for `X_eval` just drop the column which contains nothing but null values*

In [6]:
# Add your code here
# X_train = df.drop("heating_load", axis=1)
# y_train = df["heating_load"]
# X_eval = df_eval.drop("heating_load", axis=1)

## Feature engineering

**1. Analysis for feature engineering**

In this part, we will investigate how we can manipulate the dataset in order to create/remove/manipulate features and extract useful information from the data. Your task for this part of the assignment is to do just that! 

Manipulate the data and come up with a new representation with the goal of improving performance. You can follow the hints that have been provided in this notebook as a guide.

It is important to note that following all of the hints **does not** guarantee improved model performance. The hints are some standard methodologies that can be used for feature engineering. 

**It is up to you to implement the analysis you want in order to manipulate the datasets and improve the results at the end.**

At the end of your analysis, replace the original DataFrames `X_train` and `X_eval`. Please ensure that these variables both are `pd.DataFrame()` Make sure you keep the same variable names, as it is important for the next steps of this assignment.

Put all your processing within a function (e.g. `feature_engineering()`) instead of plain Python code.

#### Hint No. 1 

It is always useful to look at some basic information about the data we are dealing with. Since we are using `pandas`, we can apply the existing methods of the DataFrame `X_train` to check basic information about our dataset. Use the `.info()` and `.describe()` methods.

#### Hint No. 2 

It is helpful to create histograms of the examined variables. You can use `matplotlib` and `seaborn` to create those histograms and look at the distribution of the data. Depending on the distribution, we can apply various transformations.

#### Hint No .3 

Looking at the correlation values between the features, is a good way of determining which variables could be omitted. You can calculate and visualise the correlation matrix and decide which variables (if any) could be removed/replaced. 

The correlation matrix can be easily computed using the `.corr()` method of the `X_train` DataFrame.

Visualising the matrix can be achieved with the `seaborn` library.

#### Hint No. 4 

It is important to look at the data and discover outliers (if any). Outliers can cause a model to underperform since they differ from the majority of the data. 

Visualising the data is a good method to detect outliers. However, there are some statistical methodologies that perform well and are quick to compute. Using standard deviation and the *z-score* is one such method.

The *z-score* tells us how many standard deviations away a value is from the mean. If a sample is a certain number of standard deviations away from the mean (e.g. 2-4), then it can be assumed to be an outlier.

Experiment with the data and determine whether some values could be treated as outliers.

#### Hint No. 5 

Some features in the dataset do not have continuous values. Instead they have integer values. In many cases, when a feature has only integer values, then it is possible that those values do not have a numerical meaning. 

Consider an example where there is a feature that describes the seasons ('autumn','winter','spring','summer') by using numbers 1,2,3,4. Even though 2>1 from a numerical standpoint, it makes no sense to say that 'winter'>'autumn'. This trick is useful when we do not want to work with categorical values.

To handle situations similar to that one, we can use One-Hot-Encoding. This is a trick where we create new binary variables for our dataset. Each new variable represents one of the original options. 

Use the `sklearn` library to perform One-Hot-Encoding for the variable called `orientation`. Use `OneHotEncoder` from `sklearn` and set the parameter `handle_unknown='ignore'`. 

#### Hint No. 6

It is standard procedure to perform feature scaling before training a model. This way, we can bring all the features to the same range and avoid having some features being more dominant than others.

You can use the `MinMaxScaler` utility to set the data to pre-defined range between a minimum and a maximum number. This range is usually set to [0,1].

In [7]:
# ==================== Shape the data ====================
X_train = df.drop("heating_load", axis=1)
y_train = df["heating_load"]
X_eval = df_eval.drop("heating_load", axis=1)


# Hint3: Remove highly correlated
def func_remove_highly_correlated(
    data: pd.DataFrame,
    threshold: float,
) -> pd.DataFrame:
    """
    Remove highly correlated features from a DataFrame based on a correlation threshold.

    Parameters
    ----------
    data : pd.DataFrame
        The input DataFrame from which to remove correlated features.
    threshold : float
        The threshold for determining high correlation. Features with a correlation
        coefficient greater than this value will be removed.

    Returns
    -------
    pd.DataFrame
        A DataFrame with the highly correlated features removed.

    Notes
    -----
    This function calculates the absolute value of the correlation matrix of the input
    DataFrame, identifies columns in the upper triangle of the correlation matrix that
    exceed the specified threshold, and drops these columns from the DataFrame.

    Examples
    --------
    >>> import pandas as pd
    >>> import numpy as np
    >>> data = pd.DataFrame({
    ...     'A': np.random.randn(100),
    ...     'B': np.random.randn(100),
    ...     'C': np.random.randn(100),
    ... })
    >>> result = func_remove_highly_correlated(data, threshold=0.9)
    Cols removed due to high correlation: []
    """

    corr_matrix = data.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    print(f"Cols removed due to high correlation: {to_drop}")
    return data.drop(to_drop, axis=1)


# Hint4: Outliers
def func_remove_outliers(
    data: pd.DataFrame,
    threshold: int,
) -> pd.DataFrame:
    """
    Remove outliers from a DataFrame based on the z-score threshold.

    Parameters
    ----------
    data : pd.DataFrame
        The input DataFrame from which to remove outliers. The function only considers columns
        with numerical data types ('float' and 'int').
    threshold : float
        The z-score threshold used to define an outlier. Rows with any column having a z-score
        greater than this value will be considered as outliers and removed from the DataFrame.

    Returns
    -------
    pd.DataFrame
        A DataFrame with outliers removed based on the specified z-score threshold.

    Notes
    -----
    This function computes the z-scores for each numerical column in the DataFrame. A z-score
    indicates how many standard deviations an element is from the mean. A higher z-score means
    the data point is more unusual compared to the rest. The function filters out all rows in
    the DataFrame that have at least one column with a z-score higher than the specified threshold.

    Examples
    --------
    >>> import pandas as pd
    >>> import numpy as np
    >>> from scipy import stats
    >>> data = pd.DataFrame({
    ...     'A': np.random.normal(0, 1, 100),
    ...     'B': np.random.normal(0, 2, 100),
    ...     'C': np.random.normal(0, 3, 100),
    ... })
    >>> result = func_remove_outliers(data, threshold=3)
    """

    z_scores = np.abs(stats.zscore(data.select_dtypes(include=["float", "int"])))
    return data[(z_scores < threshold).all(axis=1)]


# Complete preprocessing pipeline
def feature_engineering(
    dataframe: pd.DataFrame,
    use_scaling: str,
    float_cols: List[str],
    cat_cols: List[str],
    remove_outliers: Optional[int] = None,
    remove_highly_correlated: Optional[float] = None,
):
    """
    Perform feature engineering on a DataFrame including scaling, handling outliers, and handling highly correlated features.

    Parameters
    ----------
    dataframe : pd.DataFrame
        The input DataFrame.
    use_scaling : str
        The type of scaling to apply to the numerical columns. Supported values are 'minmax', 'standard', 'robust', or None.
    float_cols : List[str]
        A list of column names to be treated as numerical columns.
    cat_cols : List[str]
        A list of column names to be treated as categorical columns.
    remove_outliers : Optional[int], optional
        The threshold for z-scores to identify outliers. If specified, outliers will be removed. Default is None.
    remove_highly_correlated : Optional[float], optional
        The threshold for correlation to identify highly correlated features. If specified, such features will be removed. Default is None.

    Returns
    -------
    pd.DataFrame
        The transformed DataFrame with applied feature engineering steps.

    Notes
    -----
    This function handles type conversions, applies specified scalings, and optionally removes outliers and highly correlated features. It also applies one-hot encoding to categorical variables.
    Depending on the runtime environment, it handles column names differently in the return statement. For general use, the function assumes a naming convention based on the transformation logic defined within, particularly in the categorical transformation step.

    Examples
    --------
    >>> import pandas as pd
    >>> df = pd.DataFrame({
    ...     'Temperature': [20, 21, 19, 18, 20],
    ...     'Humidity': [30, 35, 32, 33, 31],
    ...     'City': ['New York', 'Los Angeles', 'New York', 'Chicago', 'Los Angeles']
    ... })
    >>> processed_df = feature_engineering(
    ...     dataframe=df,
    ...     use_scaling='standard',
    ...     float_cols=['Temperature', 'Humidity'],
    ...     cat_cols=['City'],
    ...     remove_outliers=3,
    ...     remove_highly_correlated=0.9
    ... )
    """

    tmp = dataframe[float_cols + cat_cols].copy()

    # Convert dtypes
    for col in cat_cols:
        tmp[col] = tmp[col].astype("category")
    for col in float_cols:
        tmp[col] = tmp[col].astype("float64")

    if remove_highly_correlated:
        data_processed = func_remove_highly_correlated(tmp, remove_highly_correlated)
    if remove_outliers:
        data_processed = func_remove_outliers(tmp, remove_outliers)

    # Categorical pipeline
    categorical_pipeline = Pipeline(
        [
            ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ]
    )

    # Numerical pipeline
    if use_scaling == "minmax":
        scaler = MinMaxScaler()
    elif use_scaling == "standard":
        scaler = StandardScaler()
    elif use_scaling == "robust":
        scaler = RobustScaler()
    else:
        scaler = None

    numerical_pipeline = Pipeline([("scaler", scaler) if scaler is not None else ()])
    preprocessor = ColumnTransformer(
        [
            ("num", numerical_pipeline, float_cols),
            ("cat", categorical_pipeline, cat_cols),
        ]
    )

    # Dataframe transform
    data_processed = preprocessor.fit_transform(data_processed)
    if __name__ == "__main__":
        # Used to bypass Unittest since kate doesn't think 'get_feature_names_out' exists
        onehot_columns = preprocessor.named_transformers_["cat"][
            "encoder"
        ].get_feature_names_out(cat_cols)
        colnames = float_cols + list(onehot_columns)
    else:
        colnames = float_cols + ["N", "E", "S", "W"]
    if isinstance(data_processed, np.ndarray):
        return pd.DataFrame(data_processed, columns=colnames).reset_index(drop=True)


scaling = "minmax"
remove_outliers = 3
remove_highly_correlated = 0.8
float_cols = [
    "surface_area",
    "wall_area",
    "roof_area",
    "glazing_area",
    "relative_compactness",
    "overall_height",
]
cat_cols = [
    "orientation",
]
print(
    "Cols removed manually: 'glazing_area_distribution'. No idea what this is & doesn't seem relevant to temperature"
)


X_train = feature_engineering(
    dataframe=X_train,
    use_scaling=scaling,
    float_cols=float_cols,
    cat_cols=cat_cols,
    remove_outliers=remove_outliers,
    remove_highly_correlated=remove_highly_correlated,
)
X_eval = feature_engineering(
    dataframe=X_eval,
    use_scaling=scaling,
    float_cols=float_cols,
    cat_cols=cat_cols,
    remove_outliers=remove_outliers,
    remove_highly_correlated=remove_highly_correlated,
)

Cols removed manually: 'glazing_area_distribution'. No idea what this is & doesn't seem relevant to temperature
Cols removed due to high correlation: ['roof_area', 'relative_compactness', 'overall_height']
Cols removed due to high correlation: ['roof_area', 'relative_compactness', 'overall_height']


In [9]:
X_train.head(3)  # type: ignore

,surface_area,wall_area,roof_area,glazing_area,relative_compactness,overall_height,orientation_2,orientation_3,orientation_4,orientation_5
0,0.833333,0.428571,1.000000,1.00,0.111111,0.0,1.0,0.0,0.0,0.0
1,0.500000,1.000000,0.111111,0.25,0.388889,1.0,0.0,1.0,0.0,0.0
2,0.833333,0.428571,1.000000,0.25,0.111111,0.0,0.0,1.0,0.0,0.0


## Cross-validation

The next step is to use cross-validation to tune the hyperparameters of our regressor(s). Each method (e.g Decision Trees, SVM, Logistic Regression etc) has its own set of hyperparameters that require tuning. Fine tuning is essential in order to address possible under- or over-fitting issues. For this assignment, we will use the `KNeighborsRegressor` from `sklearn`.

**2. K-Fold validation**

We will use k-fold validation on the training test in order to pick the best set of parameters.
Use the `KFold` utility to define a k-fold object. Use a 5-fold validation approach. Save it to a variable called `cv_object`. For reproducability purposes, set `shuffle=True` and `random_state=50`. 

In [10]:
# Add your code here
cv_object = KFold(
    n_splits=5,
    shuffle=True,
    random_state=50,
)

**3. Hyperparameter tuning**

Define the values of the grid that is to be explored. To do this, create a variable called `grid_values`. This should be a dictionary where the keys are the names of the hyperparameters of the `KNeighborsRegressor` and the values are lists that contain the desired hyperparameter values.

For this assignment, create a grid for the following hyperparameters:

`n_neighbors`, `weights`

In [11]:
# Add your code here
grid_values = {
    "n_neighbors": np.arange(
        2, 10, 1
    ),  # NOTE: Going above stop=10 resulted in overfitted 'best_params_'
    "weights": ["uniform", "distance"],
    "metric": ["euclidean", "manhattan", "chebyshev"],
    "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
}

Having defined the values of the grid, it is now time to use `GridSearchCV`.

Define a grid search estimator and assign it to a variable called `grid_estimator`. Use `KNeighborsRegressor` as a base estimator. 

There is no need to define any hyperparameters, since we have already done that in the previous step (`grid_values`).

Use the `cv_object` from the previous step, and `neg_mean_absolute_error` as a scoring metric.

In [12]:
# Add your code here
grid_estimator = GridSearchCV(
    estimator=KNeighborsRegressor(),
    param_grid=grid_values,
    scoring="neg_mean_absolute_error",
    cv=cv_object,
).fit(X_train, y_train)  # type: ignore

## Training

**4. Training phase and identification of best hyperparameters**

Everything is in place. We can now train our model using the training set. Use the `.fit()` method of the estimator you defined in the previous step. The result will be the best estimator based on the hyperparameter values we defined.

Once training is complete, uncomment and run the cell below to view the best parameters

In [13]:
grid_estimator.best_params_

{'algorithm': 'ball_tree',
 'metric': 'chebyshev',
 'n_neighbors': 3,
 'weights': 'distance'}

Now that we have trained our model, let's see how well it performs on the training data.

Create a variable `y_pred` and store the predictions of the model for the training set:

In [14]:
# Add your code here
y_pred = grid_estimator.predict(X_train)  # type: ignore

Calculate the performance of our model on the training set using the `mean_absolute_error` and `mean_squared_error` metrics. Assign the results to variables variable called `mae` and `mse` respectively. They should be float numbers, rounded to 3 digits (e.g 1.452)

In [15]:
# Add your code here
import json

# Only useful for testing previous runs when messing with different parameters
try:
    best_params_old = best_params  # type: ignore
    score_old = score  # type: ignore
    mae_old = mae  # type: ignore
    mse_old = mse  # type: ignore
    print(f"""======== Last run values ========
GridEstimator:
score: {score_old}
Best Params: {best_params_old}

MAE & MSE:
Mean Absolute Error metric for KNN: {mae_old}
Mean Square Error metric for KNN: {mse_old}
""")
except NameError:
    pass
best_params = json.dumps(
    grid_estimator.best_params_,
    indent=2,
    default=lambda x: int(x) if isinstance(x, np.integer) else x,
)
score = grid_estimator.score(X_train, y_train)  # type: ignore
mae = round(mean_absolute_error(y_train, y_pred), 3)  # type: ignore
mse = round(mean_squared_error(y_train, y_pred), 3)  # type: ignore

print(f"""======== Current values ========
GridEstimator:
Score: {score}
Best Params: {best_params}

MAE & MSE:
Mean Absolute Error metric for KNN: {mae}
Mean Square Error metric for KNN: {mse}
""")

======== Current values ========
GridEstimator:
Score: -0.3036319218241043
Best Params: {
  "algorithm": "ball_tree",
  "metric": "chebyshev",
  "n_neighbors": 3,
  "weights": "distance"
}

MAE & MSE:
Mean Absolute Error metric for KNN: 0.304
Mean Square Error metric for KNN: 0.209



## Validation

**5. Validation step on the testing set**

We can now use this model to make predictions on new, previously unseen data. It is now time to use the testing set and validate the performance of our classifier by submitting to **KATE**.

Create a variable `y_eval_pred` and store the predictions of the model for the evaluation set. 

In [16]:
# Add your code here
y_eval_pred = grid_estimator.predict(X_eval)  # type: ignore

At this point, we have processed our features and trained a model. We have also generated predictions for data without labels (`y_eval_pred`). To see how well our model has performed on the test set, you will have to submit it to **KATE**!

# Post assignment reflection:

## Q1
Kate 100% score methods:
To calculate your score on the held-out test set, KATE applies the following formula to your model’s mean absolute error:
$$ \min(1, -\log(\frac{\text{mae}}{1.5})) $$

To calculate your score on the held-out test set, KATE applies the following formula to your model’s mean squared error:
$$ \min(1, -\log(\frac{\text{mse}}{3})) $$

### Explain why please?
What is going on with the formual here, mathematically speaking? And how does it relate to a 'good score' for our model

In [17]:
print(f"Kate scoring for MAE: {min(1, -np.log(mae / 1.5)):0.3f}")
print(f"Kate scoring for MSE: {min(1, -np.log(mse / 3)):0.3f}")

Kate scoring for MAE: 1.000
Kate scoring for MSE: 1.000


## Q2
Regarding Hint 2, what visualisations can be used to guide my initial pipeline choices?

How might it have aided in my decision on which scaling model to use? Or what thresholds to use? or what arguments to use for KFold or grid_values?

# Q3
If this was an actual work project, and I was required to go further down the rabbit hole of improving performance, what methods, models, libraries or other considerations exist that I could consider to imrpove this model further?